In [36]:
from credentials import *
from utils import swagger_prompt
import pandas as pd

In [63]:
df = pd.read_csv('../ConversationalCloud_QnA - Sheet1.tsv', sep='\t')
df = df.rename(columns={'Question' : 'question' ,'Answer' : 'ground_truths'})
df.head(4)

,question,ground_truths
0,как настроить кайлу в проекте?,Чтобы создать новый проект CAILA и настроить е...
1,При использовании $caila в сценарии тратятся л...,Да. При обращении к сервису классификации CAIL...
2,Вопрос от по использованию SalesForce - операт...,Происходит событие closeChat
3,"Можно ли так подменить $request в прематче, чт...",Включите эту опцию в конфигурационном файле ch...


In [64]:
answers = []
contexts = []

In [65]:
for row in df.iterrows():
    query = row[1].question
    response = swagger_prompt(query).json()['output'][0]
    answer = response['answer']
    found_contexts = response['texts']
    answers.append(answer)
    contexts.append(found_contexts)

In [81]:
df['contexts'] = contexts
df['answers'] = answers

In [83]:
df.ground_truths = df.ground_truths.map(lambda x: [f"'{x}'"])

In [88]:
df.head(4)

,question,ground_truths,contexts,answers
0,как настроить кайлу в проекте?,['Чтобы создать новый проект CAILA и настроить...,"[Как подключить канал Aimychat, Как добавить C...",Для настройки CAILA в проекте вам необходимо в...
1,При использовании $caila в сценарии тратятся л...,['Да. При обращении к сервису классификации CA...,[Обработка переполнения $session и $client в с...,"Нет, при использовании $caila в сценарии не тр..."
2,Вопрос от по использованию SalesForce - операт...,['Происходит событие closeChat'],[Остальная информация будет отправлена на вебх...,При завершении чата со стороны клиента в Sales...
3,"Можно ли так подменить $request в прематче, чт...",['Включите эту опцию в конфигурационном файле ...,[Хочешь еще раз? и совершит переход в стейт /П...,В документации не указано способа подменить $r...


In [89]:
df.to_csv('original_data_df.tsv', sep='\t', index=False)

In [56]:
import datasets

c:\Users\Ростислав\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [85]:
questions = [each for each in df.question]
ground_truths = [each for each in df.ground_truths]
answers = [each for each in df.answers]
contexts = [each for each in df.contexts]

In [86]:
g = {'question':questions, 'ground_truths':ground_truths, 'answer':answers, 'contexts':contexts}
dd = datasets.DatasetDict({'original': datasets.Dataset.from_dict(g)})
dd

DatasetDict({
    original: Dataset({
        features: ['question', 'ground_truths', 'answer', 'contexts'],
        num_rows: 28
    })
})

In [71]:
from ragas import evaluate
import os
from langchain.chat_models import ChatOpenAI
from ragas.llms import LangchainLLM
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    answer_correctness,
    context_recall,
    context_precision
)

os.environ['OPENAI_API_BASE'] = OPENAI_API_BASE
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [72]:
gpt4 = ChatOpenAI(model_name="gpt-4",timeout=100)
gpt4_wrapper = LangchainLLM(llm=gpt4)

faithfulness.llm = gpt4_wrapper
answer_relevancy.llm = gpt4_wrapper
answer_correctness.llm = gpt4_wrapper
context_recall.llm = gpt4_wrapper
context_precision.llm = gpt4_wrapper

In [87]:
faith = evaluate(
    dd['original'],
    metrics=[faithfulness],
)

faith

evaluating with [faithfulness]


100%|██████████| 2/2 [10:43<00:00, 321.81s/it]


{'faithfulness': 0.4455}

In [90]:
ans_cor = evaluate(
    dd['original'],  # showing only 5 for demonstration
    metrics=[answer_correctness],
)

ans_cor

evaluating with [answer_correctness]


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]


RuntimeError: ('Fatal error occurred while running async tasks.', PermissionDeniedError('<!DOCTYPE html>\n<!--[if lt IE 7]> <html class="no-js ie6 oldie" lang="en-US"> <![endif]-->\n<!--[if IE 7]>    <html class="no-js ie7 oldie" lang="en-US"> <![endif]-->\n<!--[if IE 8]>    <html class="no-js ie8 oldie" lang="en-US"> <![endif]-->\n<!--[if gt IE 8]><!--> <html class="no-js" lang="en-US"> <!--<![endif]-->\n<head>\n<title>Attention Required! | Cloudflare</title>\n<meta charset="UTF-8" />\n<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />\n<meta http-equiv="X-UA-Compatible" content="IE=Edge" />\n<meta name="robots" content="noindex, nofollow" />\n<meta name="viewport" content="width=device-width,initial-scale=1" />\n<link rel="stylesheet" id="cf_styles-css" href="/cdn-cgi/styles/cf.errors.css" />\n<!--[if lt IE 9]><link rel="stylesheet" id=\'cf_styles-ie-css\' href="/cdn-cgi/styles/cf.errors.ie.css" /><![endif]-->\n<style>body{margin:0;padding:0}</style>\n\n\n<!--[if g

In [91]:
ans_rel = evaluate(
    dd['original'],  # showing only 5 for demonstration
    metrics=[answer_relevancy],
)

ans_rel

evaluating with [answer_relevancy]


100%|██████████| 2/2 [01:56<00:00, 58.10s/it]


{'answer_relevancy': 0.9002}

In [92]:
con_rec = evaluate(
    dd['original'].select(range(5)),  # showing only 5 for demonstration
    metrics=[context_recall],
)

con_rec

evaluating with [context_recall]


  0%|          | 0/1 [01:02<?, ?it/s]


AttributeError: 'NoneType' object has no attribute 'lower'

In [93]:
con_prec = evaluate(
    dd['original'],  # showing only 5 for demonstration
    metrics=[context_precision],
)

con_prec

evaluating with [context_precision]


100%|██████████| 2/2 [04:24<00:00, 132.15s/it]


{'context_precision': 0.2576}

In [110]:
df['faithfulness'] = faith.to_pandas().faithfulness
df['answer_relevancy'] = ans_rel.to_pandas().answer_relevancy
df['context_precision'] = con_prec.to_pandas().context_precision


In [112]:
df.to_csv('original_data_w_metrics.tsv', sep='\t', index=False)